# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct, desc

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")
#citations.show(10)

In [5]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")
#patents.show(10)

Solution begins here:

In [6]:
#Filter out stateless entries
patents = patents.filter(patents.POSTATE!="null")
#patents.show(10)

In [7]:
#Set up aliases
total_count = citations.alias('total_count')
p_cited = patents.alias('p_cited')
p_citing = patents.alias('p_citing')

In [8]:
#Initial join
join1 = total_count.join(p_cited, total_count.CITED == p_cited.PATENT).select(
    col('CITED').alias('CITED1'),
    col('POSTATE').alias('CITED_POSTATE'),
    col('CITING').alias('CITING1')) 
#join.show(10)

In [9]:
#Second join
join2 = join1.join(p_citing, join1.CITING1 == p_citing.PATENT).select(
    col('CITED1'),
    col('CITED_POSTATE'),
    col('CITING1'),
    col('POSTATE').alias('CITING_POSTATE')
)
#join2.show(10)

In [10]:
#Align state citations
where1 = join2.where(col('CITED_POSTATE') == col('CITING_POSTATE'))
#where1.show(10)

In [11]:
#Organize state citation counts
where2 = where1.groupBy('CITING1').agg(count('CITING1').alias('SAME_STATE')).sort(desc('SAME_STATE'))
#where2.show(10)   

In [12]:
#Show the full solution
solution = where2.join(patents, patents.PATENT == where2.CITING1).select(
    col('Patent'),
    col('Gyear'),
    col('Gdate'),
    col('Appyear'),
    col('Country'),
    col('Postate'),
    col('Assignee'),
    col('Asscode'),
    col('Claims'),
    col('Nclass'),
    col('Cat'),
    col('Subcat'),
    col('Cmade'),
    col('Creceive'),
    col('Ratiocit'),
    col('General'),
    col('Original'),
    col('Fwdaplag'),
    col('Bckgtlag'),
    col('Selfctub'),
    col('Selfctlb'),
    col('Secdupbd'),
    col('Secdlwbd'),
    col('SAME_STATE')).sort(desc('SAME_STATE'))
solution.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| Patent|Gyear|Gdate|Appyear|Country|Postate|Assignee|Asscode|Claims|Nclass|Cat|Subcat|Cmade|Creceive|Ratiocit|General|Original|Fwdaplag|Bckgtlag|Selfctub|Selfctlb|Secdupbd|Secdlwbd|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|       103|
|6008204| 1999|14606|   1998| 